In [1]:
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir('/Users/veronica/Documents/GitHub/analysis')
from src import utils, credentials

In [2]:
creds = credentials.creds
engine = utils.create_sql_db(creds)

In [3]:
query = """WITH base as (
SELECT 
  c.contract_id,
  c.store_id,
  c.seller_name,
  c.plan_id,
  c.plan_purchase_price,
  pcrs_contracts.retail_cost_of_plan,
  pcrs_contracts.product_purchase_price,
  c.product_list_price,
  c.plan_sku,
  c.status,
  prod.reimbursement_amount/100.0 as reimbursement_amount,
  prod.price as price,
  CASE WHEN prod.reimbursement_amount IS NOT NULL THEN prod.reimbursement_amount/100.0 ELSE prod.price END as price_final,
  m.merchant_cut,
  p.retail_target,
  p.fixed_price,
  p.price_high,
  p.price_low
FROM analytics.contract_facts c
LEFT JOIN analytics.merchant_facts m -- get merchant_cut
ON c.store_id = m.store_id
LEFT JOIN analytics.product_facts prod -- get reimbursement amount
ON c.product_id = prod.product_id AND c.store_id = prod.store_id
LEFT JOIN analytics_pcrs.pcrs_contracts_sold AS pcrs_contracts
ON c.admin_contract_id = pcrs_contracts.contract_number
LEFT JOIN analytics.plans_pricing p
ON c.plan_id = p.plan_id
WHERE prod.price BETWEEN (p.price_low/100.0) and (p.price_high/100.0) 
AND c.status in ('expired', 'live')
)

SELECT
  b.*,
  pm.cost/100.0 as cost
FROM base b
LEFT JOIN analytics.plans_matching pm -- get cost (premium) based on price_final (reimbursement_amount/product_list_price)
ON b.plan_id = pm.plan_id AND b.plan_sku = pm.vendor_sku
WHERE 
b.price_final BETWEEN (b.price_low/100.0) AND (b.price_high/100.0)"""

df = pd.read_sql_query(query, engine)

In [11]:
# Clean Data
# Remove contracts with a fixed price, empty cost value, where cost=1, or product_price=1

In [4]:
df_clean = df[(df['cost'].notnull()) & (df['cost']!= 0.01) & 
              (df['fixed_price'].isnull()) & (df['price']!= 1)]

df_clean['retail_cost_of_plan'] = np.where(df_clean['retail_cost_of_plan'].isna(), df_clean['plan_purchase_price'], df_clean['retail_cost_of_plan'])
df_clean.head()

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,status,reimbursement_amount,price,price_final,merchant_cut,retail_target,fixed_price,price_high,price_low,cost
7,9d1321c7-330f-4d77-9dcc-a6bdd603bfb5,661964ec-5d06-4dea-bbb2-56d3b9817a98,Embr Labs,10001-misc-elec-base-replace-2y,43.99,43.99,299.00,299.00,EXTRPL24,live,299.00,299.00,299.00,0.30,0.18,NaN,29999,25000,22.40
8,9b123398-e8a5-4518-88ae-a5a04deda513,a5e80a29-c6af-446f-b569-f021e697c48c,YI,10001-misc-elec-base-replace-2y,6.49,6.49,21.49,21.49,EXTRPL18,live,21.49,21.49,21.49,0.25,0.18,NaN,2499,0,2.39
9,cebb5e9a-6e6d-4392-9336-6aa57646ee68,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-3y,169.00,169.00,749.99,749.99,EXTRPL50,live,799.99,799.99,799.99,0.35,0.26,NaN,99999,75000,88.31
10,d9ac5501-d56a-42b3-b773-d78eab855be8,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-1y,14.99,14.99,279.99,279.99,EXTRPL7,live,229.99,229.99,229.99,0.30,0.10,NaN,24999,20000,11.56
11,a4ee2423-f72f-46be-8028-1e0c9bff1cd0,661964ec-5d06-4dea-bbb2-56d3b9817a98,Embr Labs,10001-misc-elec-base-replace-2y,43.99,43.99,299.00,299.00,EXTRPL24,live,299.00,299.00,299.00,0.30,0.18,NaN,29999,25000,22.40


In [5]:
def calc_target_price(df):
    df['target_price'] = df['retail_target'] * df['price']
    return df

In [6]:
def calc_min_price(df, extend_cut):
    df[f'min_price_{extend_cut}'] = df['cost'] / (1 - df['merchant_cut'] - extend_cut)
    return df

In [7]:
def calc_diff(df, min_price_col):
    df['price_diff'] = abs(df['target_price'] - df[min_price_col])
    df['price_diff_pct'] = abs((df['target_price'] - df[min_price_col])/ df[min_price_col])
    return df

In [8]:
def find_price_floor(df, extend_cut):
    df[f'raw_price_{extend_cut}'] = df[['target_price', f'min_price_{extend_cut}']].max(axis=1)
    df[f'price_floor_{extend_cut}'] = np.where(df[f'min_price_{extend_cut}'] > df['target_price'], 1, 0)
    return df

In [9]:
def round_price(price):
    if price < 5:
        return round(price, 1) - 0.01
    if price < 10:
        return round(price/5, 1)*5 - 0.01
    if price < 50:
        return round(price) - 0.01
    if price < 100:
        return round(price/5)*5 - 0.01
    if price < 500: 
        return round(price/10)*10 - 1
    if price < 1000:
        return round(price/50)*50 - 1
    if price < 5000:
        return round(price/100)*100 - 1
    if price < 10000:
        return round(price/500)*500 - 1
    if price < 50000:
        return round(price/1000)*1000 - 1
    else:
        return np.nan

In [10]:
df_price = calc_target_price(df_clean)
df_price = calc_min_price(df_price, 0.10)
df_price = calc_diff(df_price, 'min_price_0.1')
df_price = find_price_floor(df_price, 0.10)
df_price.head()

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,status,...,fixed_price,price_high,price_low,cost,target_price,min_price_0.1,price_diff,price_diff_pct,raw_price_0.1,price_floor_0.1
7,9d1321c7-330f-4d77-9dcc-a6bdd603bfb5,661964ec-5d06-4dea-bbb2-56d3b9817a98,Embr Labs,10001-misc-elec-base-replace-2y,43.99,43.99,299.00,299.00,EXTRPL24,live,...,NaN,29999,25000,22.40,53.8200,37.333333,16.486667,0.441607,53.8200,0
8,9b123398-e8a5-4518-88ae-a5a04deda513,a5e80a29-c6af-446f-b569-f021e697c48c,YI,10001-misc-elec-base-replace-2y,6.49,6.49,21.49,21.49,EXTRPL18,live,...,NaN,2499,0,2.39,3.8682,3.676923,0.191277,0.052021,3.8682,0
9,cebb5e9a-6e6d-4392-9336-6aa57646ee68,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-3y,169.00,169.00,749.99,749.99,EXTRPL50,live,...,NaN,99999,75000,88.31,207.9974,160.563636,47.433764,0.295420,207.9974,0
10,d9ac5501-d56a-42b3-b773-d78eab855be8,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-1y,14.99,14.99,279.99,279.99,EXTRPL7,live,...,NaN,24999,20000,11.56,22.9990,19.266667,3.732333,0.193720,22.9990,0
11,a4ee2423-f72f-46be-8028-1e0c9bff1cd0,661964ec-5d06-4dea-bbb2-56d3b9817a98,Embr Labs,10001-misc-elec-base-replace-2y,43.99,43.99,299.00,299.00,EXTRPL24,live,...,NaN,29999,25000,22.40,53.8200,37.333333,16.486667,0.441607,53.8200,0


In [11]:
df_pf = df_price[df_price['price_floor_0.1']==1]

In [42]:
len(df_price)

240146

In [12]:
len(df_pf)

66285

In [43]:
66285/240146

0.27601958808391563

In [13]:
len(df_pf['store_id'].unique())

167

In [14]:
price_col = ['seller_name', 'plan_id', 'plan_purchase_price', 'retail_cost_of_plan', 'target_price', 'min_price_0.1']

In [15]:
# diff between plan purchase price and minimum price
df_pf['retail_min_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['min_price_0.1'])

df_pf['plan_diff'] = abs(df_pf['plan_purchase_price'] - df_pf['retail_cost_of_plan'])

# diff between target price and plan price
df_pf['retail_target_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['target_price'])

# diff between minimum price and target price
df_pf['min_target_diff'] = abs(df_pf['min_price_0.1'] - df_pf['target_price'])

# ratio of min price to product price
df_pf['min_target'] = df_pf['min_price_0.1']/df_pf['price']

# diff between (min price/product price) and retail target
df_pf['target_diff'] = abs(df_pf['min_target'] - df_pf['retail_target'])

In [16]:
df_pf[df_pf['seller_name']=='Shop Indoor Golf'][price_col+['retail_min_diff']].sort_values(by='retail_min_diff', ascending=False)

,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff
32467,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.00,649.00,609.6974,883.366667,234.366667
14229,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.00,649.00,609.6974,883.366667,234.366667
166520,Shop Indoor Golf,10003-golf-base-repair-3y,949.00,949.00,987.9987,1120.200000,171.200000
159963,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,549.00,549.00,518.7000,688.433333,139.433333
89699,Shop Indoor Golf,10003-golf-base-repair-3y,1099.00,1099.00,1104.9987,1227.700000,128.700000
...,...,...,...,...,...,...,...
146956,Shop Indoor Golf,B0-SGGLF-1y,229.00,229.00,145.9996,228.666667,0.333333
369580,Shop Indoor Golf,B0-SGGLF-1y,229.00,229.00,145.9996,228.666667,0.333333
222907,Shop Indoor Golf,A0-GEGEN-2y,84.99,84.99,74.9985,84.700000,0.290000
322883,Shop Indoor Golf,A0-GEGEN-2y,84.99,84.99,74.9985,84.700000,0.290000


In [17]:
df_pf['min_price_rounded'] = df_pf['min_price_0.1'].apply(round_price)
df_pf['retail_min_rounded_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['min_price_rounded'])

In [18]:
df_pf[df_pf['retail_min_rounded_diff'] < 5].sort_values(by='retail_min_rounded_diff', ascending=False)[price_col]

,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1
180105,1MOREUSA,10001-portable-audio-base-replace-2y,18.99,18.99,10.7982,13.538462
9064,1MOREUSA,10001-portable-audio-base-replace-2y,18.99,18.99,10.7982,13.538462
329732,DailySale,10001-jewelry-term-base-replace-3y,11.99,11.99,11.7000,17.325000
148432,1MOREUSA,10001-portable-audio-base-replace-2y,18.99,18.99,10.7982,13.538462
9229,1MOREUSA,10001-portable-audio-base-replace-2y,18.99,18.99,10.7982,13.538462
...,...,...,...,...,...,...
71388,JBL-US,10001-portable-audio-base-replace-1y,6.49,6.49,4.9950,6.300000
214119,JBL-US,10001-portable-audio-base-replace-1y,6.49,6.49,4.9950,6.300000
71399,T3,B0-HBGEN-3y,43.99,43.99,40.0000,43.650000
214109,Jegs High Performance,10001-jegs-autopart-base-replace-3y,10.99,10.99,10.7982,10.920000


## Filter out some cases where discpreancy between plan_purchase_price and min_price;
- Cases where plan_purchase_price is closer to target_price rather than min_price
- plan_purchase_price > than both min_price and target_price

In [19]:
df_anom = df_pf[(df_pf['retail_target_diff'] < df_pf['retail_min_rounded_diff']) & (df_pf['retail_min_rounded_diff'] > 5)].sort_values(by='min_target_diff', ascending=False)
df_anom[price_col+['min_price_rounded']].head()

,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,min_price_rounded
32467,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.0,649.0,609.6974,883.366667,899.0
14229,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,649.0,649.0,609.6974,883.366667,899.0
159963,Shop Indoor Golf,10001-misc-elec-adh-replace-3y,549.0,549.0,518.7000,688.433333,699.0
66955,AutoAnything,10001-truck-soft-cover-base-replace-1y,179.0,179.0,108.6300,272.857143,269.0
15646,AutoAnything,10001-truck-rollaway-cover-base-replace-2y,209.0,209.0,190.1200,322.371429,319.0


In [20]:
contracts_ex = df_anom['contract_id'].unique()

In [21]:
df_pf_final = df_pf[~df_pf['contract_id'].isin(contracts_ex)]
len(df_pf_final)

65071

## Impact Analysis
- how large is the difference between min_price and target_price?
- which merchants are most impacted?

In [60]:
df_pf_final['product_list_price'].sum()

8731242.47

In [22]:
df_pf_final['seller_name'].value_counts().head(20)

BlendJet                  16284
JBL-US                    10674
RealTruck                  3948
Carandtruckremotes.com     3858
L'ange Hair                3747
Skullcandy                 2601
EightVape                  2205
RaceDayQuads               1376
DailySale                  1342
Parts Dr                   1257
Jetson Electric Bikes      1125
Logitech                   1030
1MOREUSA                   1005
Jegs High Performance       924
Wyze Ecommerce              795
PCLiquidations              768
Motosport                   765
Blackstone Products         633
Renogy                      561
Backcountry.com             551
Name: seller_name, dtype: int64

In [126]:
# In most cases, difference between target and minimum price isn't significantly differents

In [23]:
df_pf_final['min_target_diff'].describe()

count    65071.000000
mean         5.344349
std         18.838173
min          0.000400
25%          0.384900
50%          0.970231
75%          3.534000
max        449.350000
Name: min_target_diff, dtype: float64

In [50]:
df_pf_final[df_pf_final['min_target_diff'] < 5] #['seller_name'].value_counts().head(10)

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,status,...,raw_price_0.1,price_floor_0.1,retail_min_diff,plan_diff,retail_target_diff,min_target_diff,min_target,target_diff,min_price_rounded,retail_min_rounded_diff
23,be94de62-9601-473e-8f99-c7f670dc6262,2a022dbf-ed7a-4d84-8807-0b917e62d36d,Zebit,10001-gaming-console-base-repair-2y,59.99,59.99,309.99,309.99,EXTGAMH34,live,...,58.550000,1,1.440000,0.0,4.1918,2.751800,0.188877,0.008877,59.99,0.000000e+00
49,cb87ae57-d145-4c97-b716-dedff1d19e93,446af1d1-22e5-41cb-bbb1-dc6e7ad970b4,1MOREUSA,10001-misc-elec-base-replace-3y,5.99,5.99,14.99,14.99,EXTRPL35,live,...,5.492308,1,0.497692,0.0,2.0926,1.594908,0.366398,0.106398,5.49,5.000000e-01
52,17c1b869-fceb-4e7c-99cf-21f527becf06,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-1y,6.49,6.49,59.99,59.99,EXTRPL3,live,...,6.416667,1,0.073333,0.0,0.4910,0.417667,0.106962,0.006962,6.49,0.000000e+00
91,98183fa8-ad72-4a3b-8d67-6cc25f7c9b73,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-1y,5.99,5.99,59.99,59.99,EXTRPL3,live,...,6.416667,1,0.426667,0.0,0.0090,0.417667,0.106962,0.006962,6.49,5.000000e-01
121,3a617cbc-1819-42d6-9adc-7e2a74454d47,2a022dbf-ed7a-4d84-8807-0b917e62d36d,Zebit,10001-misc-elec-base-replace-1y,5.99,5.99,51.99,51.99,EXTRPL3,live,...,6.416667,1,0.426667,0.0,0.7910,1.217667,0.123421,0.023421,6.49,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369911,cd35d9cd-593f-4ef7-951d-119aef5a7b52,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-misc-elec-adh-replace-3y,38.99,38.99,NaN,137.99,EXTRPL89,live,...,38.850000,1,0.140000,0.0,3.1126,2.972600,0.281542,0.021542,38.99,0.000000e+00
369914,b1bc12eb-fe0f-45db-9195-0aef873aa484,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-misc-elec-adh-replace-3y,64.99,64.99,NaN,231.99,EXTRPL91,live,...,63.025000,1,1.965000,0.0,4.6726,2.707600,0.271671,0.011671,64.99,0.000000e+00
369915,ed22d363-9d42-4ec3-a7fb-1f640300f943,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-misc-elec-adh-replace-1y,3.09,3.09,NaN,11.99,EXTRPL52,live,...,3.075000,1,0.015000,0.0,1.8910,1.876000,0.256464,0.156464,3.09,4.440892e-16
369916,54ae833c-9440-474a-afe8-ad880701c5a6,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-misc-elec-adh-replace-1y,3.09,3.09,NaN,15.97,EXTRPL52,live,...,3.075000,1,0.015000,0.0,1.4930,1.478000,0.192549,0.092549,3.09,4.440892e-16


In [51]:
52375/65000

0.8057692307692308

In [25]:
df_jetson = df_price[df_price['seller_name']=='Jetson Electric Bikes']
df_jetson[price_col]

,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1
1232,Jetson Electric Bikes,10001-scooter-base-replace-1y,49.99,49.99,16.49945,49.927273
1347,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,40.99,40.99,39.99900,28.636364
1483,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,30.99,30.99,29.99900,21.018182
1546,Jetson Electric Bikes,10001-scooter-base-replace-2y,74.99,74.99,29.99900,74.890909
1594,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,7.49,7.49,5.99900,7.000000
...,...,...,...,...,...,...
368365,Jetson Electric Bikes,10001-misc-elec-adh-replace-2y,69.99,69.99,67.99830,34.654545
368372,Jetson Electric Bikes,10001-misc-elec-base-replace-2y,9.49,9.49,8.99820,7.636364
368607,Jetson Electric Bikes,10001-electric-mobility-base-replace-2y,74.99,74.99,27.99860,74.890909
368703,Jetson Electric Bikes,10001-misc-elec-adh-replace-1y,30.99,30.99,34.99900,15.872727


In [26]:
df_jetson = calc_min_price(df_jetson, 0.05)
df_jetson = find_price_floor(df_jetson, 0.05)

In [27]:
price_col_2 = ['seller_name', 'plan_id', 'plan_purchase_price',
               'retail_cost_of_plan', 'target_price', 'min_price_0.1',
              'min_price_0.05', 'retail_target']

In [28]:
df_jetson[price_col_2]

,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,min_price_0.05,retail_target
1232,Jetson Electric Bikes,10001-scooter-base-replace-1y,49.99,49.99,16.49945,49.927273,45.766667,0.055
1347,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,40.99,40.99,39.99900,28.636364,26.250000,0.100
1483,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,30.99,30.99,29.99900,21.018182,19.266667,0.100
1546,Jetson Electric Bikes,10001-scooter-base-replace-2y,74.99,74.99,29.99900,74.890909,68.650000,0.100
1594,Jetson Electric Bikes,10001-misc-elec-base-replace-1y,7.49,7.49,5.99900,7.000000,6.416667,0.100
...,...,...,...,...,...,...,...,...
368365,Jetson Electric Bikes,10001-misc-elec-adh-replace-2y,69.99,69.99,67.99830,34.654545,31.766667,0.170
368372,Jetson Electric Bikes,10001-misc-elec-base-replace-2y,9.49,9.49,8.99820,7.636364,7.000000,0.180
368607,Jetson Electric Bikes,10001-electric-mobility-base-replace-2y,74.99,74.99,27.99860,74.890909,68.650000,0.140
368703,Jetson Electric Bikes,10001-misc-elec-adh-replace-1y,30.99,30.99,34.99900,15.872727,14.550000,0.100


In [29]:
len(df_price[df_price['seller_name']=='Jetson Electric Bikes'])

2775

In [30]:
df_jetson.columns

Index(['contract_id', 'store_id', 'seller_name', 'plan_id',
       'plan_purchase_price', 'retail_cost_of_plan', 'product_purchase_price',
       'product_list_price', 'plan_sku', 'status', 'reimbursement_amount',
       'price', 'price_final', 'merchant_cut', 'retail_target', 'fixed_price',
       'price_high', 'price_low', 'cost', 'target_price', 'min_price_0.1',
       'price_diff', 'price_diff_pct', 'raw_price_0.1', 'price_floor_0.1',
       'min_price_0.05', 'raw_price_0.05', 'price_floor_0.05'],
      dtype='object')

In [31]:
len(df_jetson[df_jetson['price_floor_0.1']==1])

1126

In [32]:
len(df_jetson[df_jetson['price_floor_0.05']==1])

1105

In [33]:
df_jetson[['raw_price_0.1', 'raw_price_0.05', 'target_price', 'min_price_0.1', 'min_price_0.05']]

,raw_price_0.1,raw_price_0.05,target_price,min_price_0.1,min_price_0.05
1232,49.927273,45.766667,16.49945,49.927273,45.766667
1347,39.999000,39.999000,39.99900,28.636364,26.250000
1483,29.999000,29.999000,29.99900,21.018182,19.266667
1546,74.890909,68.650000,29.99900,74.890909,68.650000
1594,7.000000,6.416667,5.99900,7.000000,6.416667
...,...,...,...,...,...
368365,67.998300,67.998300,67.99830,34.654545,31.766667
368372,8.998200,8.998200,8.99820,7.636364,7.000000
368607,74.890909,68.650000,27.99860,74.890909,68.650000
368703,34.999000,34.999000,34.99900,15.872727,14.550000


In [35]:
df_jetson['min_diff'] = df_jetson['min_price_0.1'] - df_jetson['min_price_0.05']

In [36]:
df_jetson['min_diff'].describe()

count    2775.000000
mean        3.239514
std         2.657718
min         0.186364
25%         1.322727
50%         2.386364
75%         4.159091
max        26.721212
Name: min_diff, dtype: float64

In [39]:
df_price_v2 = calc_min_price(df_price, 0.05)
#df_jetson = find_price_floor(df_price, 0.05)

In [40]:
df_price_v2['min_diff'] = df_price_v2['min_price_0.1'] - df_price_v2['min_price_0.05']

In [41]:
df_price_v2['min_diff'].describe()

count    240146.000000
mean          2.337743
std           4.748983
min           0.014286
25%           0.580769
50%           1.131818
75%           2.309091
max         192.919048
Name: min_diff, dtype: float64